<a href="https://colab.research.google.com/github/Ubaid-Manzoor/N-Layer-Neural-Network-Using-TensorFlow-With-Mnist-Dataset/blob/master/N_Layer_CNN(Mnist_dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.python.framework import ops
import numpy as np
import tensorflow as tf
import math
import matplotlib.pyplot as plt

np.random.seed(1)

In [0]:
mnist = tf.keras.datasets.mnist

(X_train , y_train) , (X_test , y_test) = mnist.load_data()

In [0]:
y_train = tf.one_hot(y_train , len(np.unique(y_train)))
y_test = tf.one_hot(y_test , len(np.unique(y_test)))

In [41]:
print(X_train.shape)
print(X_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [42]:
with tf.Session() as sess:
  y_train = sess.run(y_train)
  y_test = sess.run(y_test)
print(y_train.shape)
print(y_test.shape)

(60000, 10)
(10000, 10)


In [0]:
def create_placeholder(n_H , n_W , n_C , n_y):
  
  X = tf.placeholder(shape=[None , n_H, n_W, n_C] , dtype=tf.float32 , name='X')
  Y = tf.placeholder(shape=[None, n_y] , dtype=tf.float32 , name="Y")
  
  return X,Y

In [44]:
#Create a Parameters dict to Hold all Weights
#The Weight_shape has to be create manually 
parameters_shape = {}
Weight_shapes = [[4,4,3,5] , [3,3,5,10] , [2,2,10,15]]# length of Weight_Shape depends on number of layers
for i,shape in enumerate(Weight_shapes):
  l=i+1
  parameters_shape["W"+str(l)] = shape
print(parameters_shape)

#create dict for non learnable parameters Like stride and max_pool Window size
non_learn_para = {}

stride = [[1,4] , [1,3] , [1,2]] # [[stride of conv layer , stride for max pool layer],[],[]]
window_size = [4,3,2] #[for first maxpool layer,for second,for third]

for l in range(1,len(window_size)+1):
  non_learn_para["Conv_s"+str(l)] = stride[l-1][0]
  non_learn_para["Maxpool_s"+str(l)] = stride[l-1][1]
  non_learn_para["WS"+str(l)] = window_size[l-1]
print(non_learn_para)

{'W1': [4, 4, 3, 5], 'W2': [3, 3, 5, 10], 'W3': [2, 2, 10, 15]}
{'Conv_s1': 1, 'Maxpool_s1': 4, 'WS1': 4, 'Conv_s2': 1, 'Maxpool_s2': 3, 'WS2': 3, 'Conv_s3': 1, 'Maxpool_s3': 2, 'WS3': 2}


In [0]:
def initialize_parameters(shapes):
  tf.set_random_seed(1)
  L = len(shapes)+1
  parameters = {}
  for l in range(1,L):
    parameters["W" + str(l)] = tf.get_variable("W"+str(l) ,shapes["W"+str(l)],initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    
  return parameters

In [46]:
tf.reset_default_graph()
with tf.Session() as sess_test:
  parameters = initialize_parameters(parameters_shape)
  init = tf.global_variables_initializer()
  sess_test.run(init)
  print("W1 = " + str(parameters["W1"].eval().shape))
  print("W2 = " + str(parameters["W2"].eval().shape))
  print("W2 = " + str(parameters["W3"].eval().shape))

W1 = (4, 4, 3, 5)
W2 = (3, 3, 5, 10)
W2 = (2, 2, 10, 15)


In [0]:
#dimension of Fully Connected Layer
FC_layer_dims = [10]  #Now i just used 1 FC layer there can be many

In [0]:
def first_half(X , parameters , non_learn_para):
  NLP = non_learn_para
  L = len(parameters)
  for l in range(1,L+1):
    P = X
    #Conv layer
    s = NLP["Conv_s"+str(l)] 
    Z = tf.nn.conv2d(P , parameters["W"+str(l)], strides=[1,s,s,1] , padding='SAME')
    #Relu
    A = tf.nn.relu(Z)
    #MaxPool
    f = NLP["WS"+str(l)]
    s = NLP["Maxpool_s"+str(l)]
    P = tf.nn.max_pool(A , ksize=[1,f,f,1] , strides= [1,s,s,1] , paddind='SAME')
    print(P)
  return P

In [0]:
def second_half(P , layer_dims):
  L = len(layer_dims)
  for l in range(1,L+1):
    Z = tf.contrib.layers.fully_connected(P , layer_dims[l] , activation_fn_None)
  
  return Z

In [0]:
def forward_propagation(X , parameters,non_learn_para):
  
  #run_first_half
  P = first_half(X,parameters , non_learn_para)
  #Flatten
  P_flatten = tf.conterb.layers.flatten(P)
  #run_second_half
  Z = second_half(P , FC_layer_dims)
  
  return Z

In [0]:
def compute_cost(Z , Y):
  
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z , labels=Y))
  
  return cost

In [0]:
def model(X_train , Y_train , X_test , Y_test , learning_rate=0.001 
          , num_epochs=50 , minibatch_size=64 , print_cost = True):
  
  tf.reset_default_graph()
  tf.set_random_seed(1)
  
  m , n_H0 , n_W0 , n_C0 = X_train.shape
  n_y = Y_train.shape[1]
  
  